In [7]:
import nbimporter
import time
from crypto_watch import crypto_watch
import poloniex_machine

In [8]:
class Ledger:
    exchanges = []
    coins = []
    
    def __init__(self, coin, base, quote, price, amount):
        self.data = {'coin' : coin, 
                     'base' : base, 
                     'quote' : quote, 
                     'price' : price, 
                     'amount' : amount, 
                     'profit' : None, 
                     'status' : 'o-o'}
       
        self.str_data = {'coin' : coin,
                     'base' : base, 
                     'quote' : quote, 
                     'price' : price, 
                     'amount' : amount, 
                     'profit' : None, 
                     'status' : 'o-o'}
    
    def __call__(self):
        return self.str_data
    
    def set_status(self, status):
        self.data['status'] = status
        self.str_data['status'] = status

        
    def compute_profit(self, book_base, book_quote):
        '''
        - 해당 장부의 프리미엄과 거래수행 했을때 profit 을 반환한다.
        - 2 차 orderbook data 까지 받아온다.
        '''
        taker_percent = .25
        total_fee = taker_percent * 3
        bid = book_base # base 에서 bid 를 받음
        ask = book_quote # quote 에서 ask 를 받음
        bid_price_1 = bid[0][0][0][0]
        bid_amount_1 = bid[0][0][0][1]
        ask_price_1 = ask[0][0][0][0]
        ask_amount_1 = ask[0][0][0][1]
        bid_price_2 = bid[0][0][1][0]
        bid_amount_2 = bid[0][0][1][1]
        ask_price_2 = ask[0][0][1][0]
        ask_amount_2 = ask[0][0][1][1]
        
        premier = (bid_price_1 - ask_price_1) / ask_price_1 * 100
        if(premier < 0):
            return None

        if(ask_price_1 * ask_amount_1 < self.data['amount']):
            if(ask_price_1 * ask_amount_1 + ask_price_2 * ask_amount_2 < self.data['amount']):
                #print('> 물량부족으로 인한 거래불가')
                #print('-'*50, '\n')
                return None
            else:
                #print('* 2차 매도 호가 거래')
                total_fee += taker_percent

        if(bid_price_1 * bid_amount_1 < self.data['amount']):
            if(bid_price_1 * bid_amount_1 + bid_price_2 * bid_amount_2 < self.data['amount']):
                #print('> 물량부족으로 인한 거래불가')
                #print('-'*50, '\n')
                return None
            else:
                #print('* 2차 매수 호가 거래')
                total_fee += taker_percent

        profit = premier - total_fee
        if(profit < 0):
            #print('> taker 수수료로 인한 손익분기점 넘지 못함.')
            #print('-'*50, '\n')
            return profit

        #print('매도호가 -', coin, exchanges[self.data['base']], ask, '- 총액:', round(amount_ask, 3), 'USD')
        #print('매수호가 -', coin, exchanges[self.data['quote']], bid, '- 총액:', round(amount_bid, 3), 'USD')
        #print('실질적 프리미엄 :', round(premier, 3), '%')
        print('획득가능 이윤 :', round(profit, 3), '%')
        print('예상 수익금 :', self.data['amount'] * round(profit, 3) / 100, 'USD', '\n')
        
        #print('-'*50, '\n')
        self.data['profit'] = profit
        self.str_data['profit'] = profit
        return profit

In [9]:
class Balance:

    def __init__(self, exchange):
        self.data = {'exchange' : exchange, 
                     'fiat' : None, 
                     'coins' : {}}
    
    def set_balance(self):
        self.data['coins']['btc'] = 1
        self.data['coins']['eth'] = 2
    
    def __call__(self):
        return self.data

In [13]:
class Trader:
    FEE = {'poloniex' : .25, 
           'kraken' : .25}
    
    def __init__(self, initial_balance, exchanges=[], coins=[], max_channels=5, cryptowatch=None):
        self.managing_balance = []
        self.arbi_ledger = [] # 차익거래 정보가 담긴 장부
        self.portfolio_history = []
        self.open_orders = [] # order class 새로 정의할까??
        self.exchanges = exchanges
        self.coins = coins
        self.machines = []
        self.initial_balance = initial_balance # 최초 총 투입 자본
        self.max_channels = max_channels # 거래소 당 운용할 재정거래 최대 채널 수
        self.cur_channels = {} # 현재 거래소당 활성화된 채널
        self.cur_available_fiat = {} # 현재 가용한 현금
        self.moving_amount = 0 # 1 회 운용될 때 움직이는 금액 (20 % 는 여유금으로 남김)
        self.switching = False
        self.cw = cryptowatch
        
        # 테스트
        #self.call_public_api()
        
    def set_exchanges(self, exchanges):
        self.exchanges = exchanges
        Ledger.exchanges = exchanges
        self.cur_channels = dict([(exc, 0) for exc in self.exchanges])
        self.cur_available_fiat = dict([(exc, self.initial_balance / 3) for exc in self.exchanges])
        self.moving_amount = self.initial_balance * .8 / (self.max_channels * len(self.exchanges))
    
    def set_coins(self, coins):
        self.coins = coins
        Ledger.coins = coins
        
    def set_machines(self):
        self.machine.append(poloniex_machine.PoloMachine())
        
    def add_machines(self):
        self.machine.append(poloniex_machine.PoloMachine())
        
    def get_exchanges(self):
        return self.exchanges
    
    def get_coins(self):
        return self.coins
    
    def get_initial_balance(self):
        return self.initial_balance
    
    def get_status(self):
        return self.managing_balance, self.arbi_ledger, self.portfolio_history
    
    def call_public_api(self):
        start = time.time()
        self.cw.call_api()
        #self.data = self.cw.get_orderbooks(pairs = coins, exchanges = exchanges, limit=2) # orderbook 데이터
        print('api call 소요시간 :', round(time.time() - start, 3), '초')
    
    #----------------- trading ------------------
    
    def is_in_ledger(self, coin, base, quote):
        '''
        - 거래소 exchange 가 ledger 에 포함되어있는지 체크.
        - 같은 건수 중복 거래하는것을 방지하기 위함.
        '''
        
        for ledger in self.arbi_ledger:
            if(ledger()['base'] == base and ledger()['quote'] == quote and ledger()['coin'] == coin):
                return True
        return False
    
    
    def compute_premier(self, coin, book_base, book_quote):
        '''
        - 장부에 추가할 목적으로 프리미엄을 계산한다. (대략의 가격차이만 계산한다.)
        - 프리미엄이 0 에 가까울 때를 포착하고 ledger 에 추가하기 위함.
        '''
        
        ask = book_base
        bid = book_quote
        ask_price = ask[0][0][0][0]
        bid_price = bid[0][0][0][0]
        premier = None
        
        if(ask < bid):
            premier = (bid_price - ask_price) / ask_price * 100
            return premier
        else:
            premier = (ask_price - bid_price) / bid_price * 100
            return premier
        
        return premier

                
    def streaming(self):
        '''
        - main loop 에서 돌아갈 메서드.
        - 장부에 없는 종목에 한해 프리미엄을 계산하고 차이가 없으면 장부에 추가
        - 장부에 있는 건수들에 대해 프리미엄을 계산하고 조건이 맞으면 재정거래 수행
        - 반복
        '''
        
        # public api call 필요.
        #self.call_public_api()
        
        # ledger 에 추가하는 부분
        print('\n\n================ Loop ==================')
        print('\n--------- 장부 추가 건 -----------')
        ledgers_to_add = []
        for i_c, coin in enumerate(self.coins):
            for i_base, base in enumerate(self.exchanges):
                for i_quote, quote in enumerate(self.exchanges):
                    if(base == quote):
                        continue
                    if(self.is_in_ledger(coin, base, quote)):
                        #print('!! 장부에 중복 !!', coin, base, quote)
                        continue
                    if(self.cur_channels[base] >= self.max_channels or self.cur_channels[quote] >= self.max_channels):
                        #print('!! 채널 부족 !!')
                        continue

                    book_base = self.cw.get_orderbooks([base], [coin])[0] # 이 떄는 base 에서 ask 가 필요
                    book_quote = self.cw.get_orderbooks([quote], [coin])[1] # qoute 에서 bid 가 필요
                    
                    premier = self.compute_premier(coin, book_base, book_quote)
                    
                    if(premier < .1):
                        self.cur_channels[base] += 1
                        self.cur_channels[quote] += 1
                        self.buy_order(coin, base, book_base[0][0][0][0], self.moving_amount) # base 에서 매수 / 수수료 고려하기!
                        ledgers_to_add.append((coin, base, quote, book_base[0][0][0][0], self.moving_amount))
                        print('> 장부 추가', round(premier, 3), '%', '--', coin, base, quote, '-', book_base[0][0][0][0], self.moving_amount)

        # 루프 다 돌고나서 추가
        for ledger in ledgers_to_add:
            self.add_ledger(ledger[0], ledger[1], ledger[2], ledger[3], ledger[4])
            #print('> 장부 추가', ledger[0], ledger[1], ledger[2], ledger[3], ledger[4])
        
        # ledger 에서 거래 조건 찾고 거래하는 부분
        print('\n--------- 재정거래 건 -----------')
        ledgers_to_remove = []
        for ledger in self.arbi_ledger:
            book_base = self.cw.get_orderbooks([ledger()['base']], [ledger()['coin']], limit=2)[1] # 이 때는 base 에서 bid 가 필요, 2 차 호가까지 넘김
            book_quote = self.cw.get_orderbooks([ledger()['quote']], [ledger()['coin']], limit=2)[0]  # quote 에서 ask 가 필요
            profit = ledger.compute_profit(book_base, book_quote)
            
            if(profit != None and profit > .1):
                self.excute_trade(ledger, book_base, book_quote)
                ledgers_to_remove.append(ledger)
                print('> 재정거래 수행', 'profit :', round(profit, 3), '%')
            elif(profit != None):
                print('> 조건 미충족', round(profit, 3), ledger())
                
        # 루프 다 돌고나서 삭제
        for ledger in ledgers_to_remove:
            self.arbi_ledger.remove(ledger)
            print('> 장부 삭제', ledger())
        
        # 출력용
        print('\n--- 장부 리스트 ---')
        for ledger in self.arbi_ledger:
            print(ledger.str_data)
            
        print('\n--- 잔고 리스트 ---')
        
        print('\n--- 내역 리스트 ---')
        for history in self.portfolio_history:
            print(history)
            
        self.print_account()
        
        
    def add_ledger(self, coin, base, quote, price, amount):
        '''
        - ledger 에 건수 추가하는 메서드.
        '''
        self.cur_available_fiat[base] -= amount
        self.cur_available_fiat[quote] -= amount * 1.1
        self.arbi_ledger.append(Ledger(coin, base, quote, price, amount))

        
    def update_balance(self):
        pass
    
    def excute_trade(self, ledger, book_base, book_quote):
        '''
        - 실질적으로 거래를 수행하는 메서드.
        '''
        self.sell_order(ledger()['coin'], ledger()['base'], book_base[0][0])
        self.buy_order(ledger()['coin'], ledger()['quote'], book_quote[0][1])
        
        self.cur_channels[ledger()['base']] -= 1
        self.cur_channels[ledger()['quote']] -= 1
        self.cur_available_fiat[ledger()['base']] += (ledger()['amount'] + ledger()['amount'] * ledger()['profit']) # 실제 매도한 만큼 추가 = 이익금 + 투자금
        self.cur_available_fiat[ledger()['quote']] += (ledger()['amount'] * .1) # 묶었던 금액 - 매수하는 데 사용된 금액
        self.portfolio_history.append(ledger.str_data)

        
    def buy_order(self, coin, exchange, price, amount):
        print('> 매수', coin, exchange)
    
    def sell_order(self, coin, exchange):
        print('> 매도', coin, exchange)
    
    def cancel_order(self):
        print('> 취소')
        
    def print_account(self):
        print('보유 현금 -', end=' ')
        for i, exc in enumerate(self.exchanges):
            print(exc, self.cur_available_fiat[exc], sep=' : ',end=', ')
            
    
    # 미체결 거래를 확인해야할 듯.
    def check_open_orders(self):
        pass
    
    def __repr__(self):
        return '>> Trader <<'

In [14]:
"""
테스트 Cell
"""
cw = crypto_watch()
exchanges = ['poloniex', 'bitfinex', 'kraken']
coins = ['xrp', 'qtum', 'xmr', 'eos', 'etc', 'bch']
#cw.call_api()
#cw.get_prices(exchanges[:2], coins[:1])
trader = Trader(initial_balance=1000, cryptowatch = cw)

In [15]:
trader.set_exchanges(exchanges)
trader.set_coins(coins)
while(True):
    trader.streaming()



================ Loop ==================

--------- 장부 추가 건 -----------
> 매수 xrp poloniex
> 장부 추가 0.064 % -- xrp poloniex kraken - 0.27239511 53.333333333333336
> 매수 xrp kraken
> 장부 추가 0.005 % -- xrp kraken poloniex - 0.27241 53.333333333333336
> 매수 xmr poloniex
> 장부 추가 0.078 % -- xmr poloniex bitfinex - 81.75510204 53.333333333333336
> 매수 xmr kraken
> 장부 추가 0.013 % -- xmr kraken bitfinex - 81.68 53.333333333333336
> 매수 eos kraken
> 장부 추가 0.077 % -- eos kraken bitfinex - 3.6364 53.333333333333336
> 매수 bch kraken
> 장부 추가 0.035 % -- bch kraken bitfinex - 312.4 53.333333333333336

--------- 재정거래 건 -----------

--- 장부 리스트 ---
{'coin': 'xrp', 'base': 'poloniex', 'quote': 'kraken', 'price': 0.27239511, 'amount': 53.333333333333336, 'profit': None, 'status': 'o-o'}
{'coin': 'xrp', 'base': 'kraken', 'quote': 'poloniex', 'price': 0.27241, 'amount': 53.333333333333336, 'profit': None, 'status': 'o-o'}
{'coin': 'xmr', 'base': 'poloniex', 'quote': 'bitfinex', 'price': 81.75510204, 'amount': 53.3

TypeError: 'NoneType' object is not subscriptable